<a href="https://colab.research.google.com/github/Yuchen686/parking-spaces/blob/main/George_parking_spaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from math import factorial, log, exp
import matplotlib.pyplot as plt

from math import log, exp

def erlang_b(lambda_rate, mu_rate, num_servers):
    '''
    Calculate the probability that all service facilities are occupied under the M/M/c/c model
    (Erlang-B formula), using logarithms for numerical stability.
    Parameters:
    - lambda_rate: arrival rate (scalar)
    - mu_rate: service rate (scalar)
    - num_servers: number of parking spaces (int)
    Returns:
    - P_full: probability of all parking spaces being occupied
    '''
    rho = lambda_rate / mu_rate  # Flow intensity
    log_rho = log(rho)  # Logarithm of rho

    # Calculate the numerator (logarithmic version)
    log_numerator = num_servers * log_rho - log_factorial(num_servers)

    # Calculate the denominator (logarithmic summation)
    log_denominator_terms = [
        k * log_rho - log_factorial(k) for k in range(num_servers + 1)
    ]
    log_denominator = log_sum_exp(log_denominator_terms)

    # Calculate P_full using the difference of logs
    log_p_full = log_numerator - log_denominator
    P_full = exp(log_p_full)  # Exponentiate back to get the final value
    return P_full

def log_factorial(n):
    '''Compute the logarithm of factorial using summation of logs.'''
    return sum(log(k) for k in range(1, n + 1))

def log_sum_exp(log_values):
    '''Compute log(sum(exp(log_values))) in a numerically stable way.'''
    max_log = max(log_values)
    return max_log + log(sum(exp(val - max_log) for val in log_values))

def find_min_parking(lambda_rate, mu_rate, target_probability):
    '''
    Find the minimum number of parking spaces required to meet the target full probability.
    Parameters:
    - lambda_rate: arrival rate (scalar)
    - mu_rate: service rate (scalar)
    - target_probability: target probability (e.g., 0.005 for 0.5%)
    Returns:
    - Minimum number of parking spaces (int)
    '''
    num_servers = 1
    while True:
        P_full = erlang_b(lambda_rate, mu_rate, num_servers)
        if P_full < target_probability:
            return num_servers
        num_servers += 1

# Generate exponential distribution samples
lambda_rate_exp_samples = np.random.exponential(scale=10, size=1000)  # Generate arrival rates
mu_rate_exp_samples = np.random.exponential(scale=10, size=1000) # Generate departure rates
# Other parameters
target_probability = 0.005  # Target: less than 0.5% probability of full parking

# Calculate the minimum number of parking spaces for each value in the distribution
parking_spaces = [
    find_min_parking(lambda_rate, mu_rate, target_probability)
    for lambda_rate, mu_rate in zip(lambda_rate_exp_samples, mu_rate_exp_samples)
]

print(f"Max parking spaces required: {max(parking_spaces)}")
print(f"Min parking spaces required: {min(parking_spaces)}")
print(f"Average parking spaces required: {np.mean(parking_spaces):.2f}")

# Visualization: Scatter plot of required parking spaces
plt.figure(figsize=(10, 6))
plt.scatter(lambda_rate_exp_samples, mu_rate_exp_samples, c=parking_spaces, cmap='viridis', alpha=0.7, edgecolor='k')
plt.colorbar(label='Minimum Parking Spaces')
plt.xscale('log')
plt.yscale('log')
plt.title('Minimum Parking Spaces vs. Arrival and Departure Rates')
plt.xlabel('Arrival Rate (λ)')
plt.ylabel('Departure Rate (μ)')
plt.grid()
plt.show()